### Tracebacks

Un traceback es el cuerpo del texto que puede apuntar al origen (y al final) de un error no controlado.

In [1]:
open("/path/to/mars.jpg")

FileNotFoundError: [Errno 2] No such file or directory: '/path/to/mars.jpg'

(consultar open.py)

### Controlando las excepciones

Cuando encuentres por primera vez excepciones que muestren tracebacks grandes como salida, es posible que te veas tentado a detectar todos los errores para evitar que esto suceda.

Si estás en una misión a Marte, ¿Qué podrías hacer si un texto del sistema de navegación dice 'Se ha producido un error'? Imagina que no hay ninguna más información ni contexto, simplemente una luz roja parpadeante con el texto del error. Como desarrollador, resulta útil ponerse al otro lado del programa: ¿Qué puede hacer un usuario cuando se produce un error?

Aunque en este módulo se explica cómo controlar las excepciones detectándolas, no es necesario detectar las excepciones todo el tiempo. A veces resulta útil permitir que se puedan generar excepciones para que otros autores de llamadas puedan tratar los errores.

#### Try y Except

Sabemos que, si no existe un archivo o directorio, se genera FileNotFoundError. Si queremos controlar esa excepción, podemos hacerlo con un bloque try y except

In [1]:
try:
    open('config.txt')
except FileNotFoundError:
    print("No se logro encontrar el archivo config.txt!")

No se logro encontrar el archivo config.txt!


Después de la palabra clave try, agregamos código que tenga la posibilidad de producir una excepción. A continuación, agregamos la palabra clave except junto con la posible excepción, seguida de cualquier código que deba ejecutarse cuando se produce esa condición. Puesto que config.txt no existe en el sistema, Python imprime que el archivo de configuración no está ahí. El bloque try y except, junto con un mensaje útil, evita un seguimiento y sigue informando al usuario sobre el problema.

Si necesitas acceder al error asociado a la excepción, debes actualizar la línea except para incluir la palabra clave as. Esta técnica es práctica si una excepción es demasiado genérica y el mensaje de error puede ser útil

In [2]:
try:
    open("mars.jpg")
except FileNotFoundError as err:
    print("Hay un problema al intentar leer el archivo:", err)

Hay un problema al intentar leer el archivo: [Errno 2] No such file or directory: 'mars.jpg'


En este caso, as err significa que err se convierte en una variable con el objeto de excepción como valor. Después, usa este valor para imprimir el mensaje de error asociado a la excepción. Otra razón para usar esta técnica es acceder directamente a los atributos del error. 

Por ejemplo, si detecta una excepción OSError más genérica, que es la excepción primaria de FilenotFoundError y PermissionError, podemos diferenciarlas mediante el atributo .errno

In [3]:
try:
    open("config.txt")
except OSError as err:
    if err.errno == 2:
        print("No se logro encontrar el archivo config.txt!")
    elif err.errno == 13:
        print("Se encontro el archivo config.txt, pero no puede leerse")

No se logro encontrar el archivo config.txt!


Intenta usar siempre la técnica que proporcione la mejor legibilidad para el código y que ayude a mantenerlo en el futuro. A veces es necesario usar código menos legible para ofrecer una mejor experiencia de usuario cuando se produce un error.

(consultar config.py)

### Generación de excepciones

Resulta útil generar excepciones que permitan que otro código comprenda cuál es el problema.

La generación de excepciones también puede ayudar en la toma de decisiones para otro código. Como hemos visto antes, en función del error, el código puede tomar decisiones inteligentes para resolver, solucionar o ignorar un problema.

Ejemplo: Los astronautas limitan su uso de agua a unos 11 litros al día. Vamos a crear una función que, con base al número de astronautas, pueda calcular la cantidad de agua quedará después de un día o más

In [17]:
def waterLeft1(astro, waterL, daysL):
    dailyU = astro * 11
    totalU = dailyU * daysL
    totalWL = waterL - totalU
    return f"El agua total que queda después de {daysL} dias son {totalWL} litros"

In [18]:
waterLeft1(5, 100, 2)

'El agua total que queda después de 2 dias son -10 litros'

Esto no es muy útil, ya que una carencia en los litros sería un error.

Después, el sistema de navegación podría alertar a los astronautas que no habrá suficiente agua para todos en dos días. Si eres un ingeniero que programa el sistema de navegación, podrías generar una excepción en la función water_left() para alertar de la condición de error

In [19]:
def waterLeft2(astro, waterL, daysL):
    dailyU = astro * 11
    totalU = dailyU * daysL
    totalWL = waterL - totalU
    
    if totalWL < 0: # Si el agua total es menos a 0 litros
        raise RuntimeError(f"No hay suficiente agua para {astro} astronautas para después de {daysL} dias!")
    return f"Agua total que queda después de {daysL} dias son {totalWL} litros"

In [20]:
waterLeft2(5, 100, 2)

RuntimeError: No hay suficiente agua para 5 astronautas para después de 2 dias!

La función water_left() también se puede actualizar para evitar el paso de tipos no admitidos. Intenta pasar argumentos que no sean enteros para comprobar la salida de error

In [22]:
waterLeft2("3", "200", None)

TypeError: can't multiply sequence by non-int of type 'NoneType'

El error de TypeError no es muy descriptivo en el contexto de lo que espera la función. Actualizaremos la función para que use TypeError, pero con un mensaje mejor

In [23]:
def waterL3(astro, waterL, daysL):
    for argument in [astro, waterL, daysL]:
        try: # Si el argumento es un int, entra aqui
            argument / 10
        except TypeError: # TypError se generará solo si no es del tipo correcto
            # Genera la misma excepción pero con un mejor mensaje de error
            raise TypeError(f"Todos los argumentos deben ser de tipo int, pero recibi: '{argument}'")
    daily_usage = astro * 11
    total_usage = daily_usage * daysL
    totalWL = waterL - total_usage
    if totalWL < 0:
        raise RuntimeError(f"No hay suficiente agua para {astro} astronautas para después de {daysL} dias!")
    return f"Agua total que queda después de {daysL} dias son {totalWL} litros"

In [24]:
waterL3("3", "200", None)

TypeError: Todos los argumentos deben ser de tipo int, pero recibi: '3'